In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.0.1


In [2]:
# read training data
# report_id - ID to identify report
# report_params - number of parameters to execute report (when more params specified - report will be generated faster)
# day_part - when report is executed (morning, midday or afternoon) - there is less load in the morning and in the afternoon reports are generated slower
# exec_time - time spent to produce report

column_names = ['report_id','report_params','day_part','exec_time']
raw_dataframe = pd.read_csv('train/local-test/test_dir/input/report_exec_times.csv')
dataframe = raw_dataframe.copy()

dataframe.head()

,report_id,report_params,day_part,exec_time
0,1,5,2,380
1,1,4,3,450
2,1,8,2,380
3,1,6,1,320
4,1,2,2,391


In [3]:
# report_id and day_part are categorical features. This means we need to encode these two attributes

report_id = dataframe.pop('report_id')
day_part = dataframe.pop('day_part')

In [4]:
# Encoding categorical attributes (creating as many columns as there are unique values and assigning 1 for the column from current row value)

dataframe['report_1'] = (report_id == 1)*1.0
dataframe['report_2'] = (report_id == 2)*1.0
dataframe['report_3'] = (report_id == 3)*1.0
dataframe['report_4'] = (report_id == 4)*1.0
dataframe['report_5'] = (report_id == 5)*1.0

dataframe['day_morning'] = (day_part == 1)*1.0
dataframe['day_midday'] = (day_part == 2)*1.0
dataframe['day_afternoon'] = (day_part == 3)*1.0

dataframe.head()

,report_params,exec_time,report_1,report_2,report_3,report_4,report_5,day_morning,day_midday,day_afternoon
0,5,380,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4,450,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,8,380,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6,320,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2,391,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
# Splitting training dataset into train (80%) and test data

train_dataset = dataframe.sample(frac=0.8,random_state=0)
test_dataset = dataframe.drop(train_dataset.index)

train_dataset.shape

(1200, 10)

In [6]:
test_dataset.shape

(300, 10)

In [7]:
# Describe train dataset, without target feature - exec_time. Mean and std will be used to normalize training data

train_stats = train_dataset.describe()
train_stats.pop("exec_time")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
report_params,1200.0,4.914167,3.159924,0.0,2.0,5.0,8.0,10.0
report_1,1200.0,0.191667,0.393776,0.0,0.0,0.0,0.0,1.0
report_2,1200.0,0.210000,0.407478,0.0,0.0,0.0,0.0,1.0
report_3,1200.0,0.192500,0.394428,0.0,0.0,0.0,0.0,1.0
report_4,1200.0,0.201667,0.401412,0.0,0.0,0.0,0.0,1.0
report_5,1200.0,0.204167,0.403259,0.0,0.0,0.0,0.0,1.0
day_morning,1200.0,0.360000,0.480200,0.0,0.0,0.0,1.0,1.0
day_midday,1200.0,0.320833,0.466991,0.0,0.0,0.0,1.0,1.0
day_afternoon,1200.0,0.319167,0.466348,0.0,0.0,0.0,1.0,1.0


In [9]:
train_stats_mean = train_stats['mean']
#print('train_stats_mean:', train_stats_mean)
train_stats_mean

report_params    4.914167
report_1         0.191667
report_2         0.210000
report_3         0.192500
report_4         0.201667
report_5         0.204167
day_morning      0.360000
day_midday       0.320833
day_afternoon    0.319167
Name: mean, dtype: float64

In [10]:
train_stats_std = train_stats['std']
train_stats_std

report_params    3.159924
report_1         0.393776
report_2         0.407478
report_3         0.394428
report_4         0.401412
report_5         0.403259
day_morning      0.480200
day_midday       0.466991
day_afternoon    0.466348
Name: std, dtype: float64

In [ ]:
# Remove exec_time feature from training data and keep it as a target for both training and testing

train_labels = train_dataset.pop('exec_time')
test_labels = test_dataset.pop('exec_time')

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
# Neural network learns better, when data is normalized (features look similar to each other)

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

normed_train_data = np.array(normed_train_data)
normed_test_data = np.array(normed_test_data)

In [ ]:
# Construct neural network with Keras API on top of TensorFlow. SGD optimizer and 
# mean squared error loss to check training quality

def build_model():
  model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
EPOCHS = 1000

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split=0.2, batch_size=40, verbose=1, callbacks=[early_stop])

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [ReportTime]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$ReportTime^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,50])
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f} Report Execution Time".format(mae))

In [ ]:
# Construct test data row with "unseen" feature values - report_params = 15 

headers = ['report_id', 'report_params', 'day_part']
dataset_input = pd.DataFrame([[1, 15, 3]],
                                columns=headers, 
                                dtype=float,
                                index=['input'])

print(dataset_input)

In [ ]:
# Encode categorical features for test data row

report_id = dataset_input.pop('report_id')
day_part = dataset_input.pop('day_part')

dataset_input['report_1'] = (report_id == 1)*1.0
dataset_input['report_2'] = (report_id == 2)*1.0
dataset_input['report_3'] = (report_id == 3)*1.0
dataset_input['report_4'] = (report_id == 4)*1.0
dataset_input['report_5'] = (report_id == 5)*1.0

dataset_input['day_morning'] = (day_part == 1)*1.0
dataset_input['day_midday'] = (day_part == 2)*1.0
dataset_input['day_afternoon'] = (day_part == 3)*1.0

dataset_input.tail()

In [ ]:
# Normalize test data row

normed_dataset_input = norm(dataset_input)
normed_dataset_input = np.array(normed_dataset_input)

print(normed_dataset_input)

In [ ]:
# Prediction result for report execution time. This is correct, model learns the rule correct - more report params, means execution time will be less

res = model.predict(normed_dataset_input)
print(res)

In [ ]:
import calendar;
import time;
ts = calendar.timegm(time.gmtime())

tf.saved_model.save(model, "./model_report_exec_time/" + str(ts))